Applied from [Dynamic Programming post](http://blog.refdash.com/dynamic-programming-tutorial-example/)

In [1]:
def canStopRecursive(runway, initSpeed, startIndex = 0):
  # negative base cases need to go first
  if (startIndex >= len(runway) or startIndex < 0 or initSpeed < 0 or not runway[startIndex]):
    return False
  # base case for a stopping condition
  if initSpeed == 0:
    return True
  # Try all possible paths
  for adjustedSpeed in [initSpeed, initSpeed - 1, initSpeed + 1]:
    # Recurrence relation: If you can stop from any of the subproblems,
    # you can also stop from the main problem
    if canStopRecursive(runway, adjustedSpeed, startIndex + adjustedSpeed):
      return True
  return False

In [2]:
def canStopIterative(runway, initSpeed, startIndex = 0):
  # maximum speed cannot be larger than length of the runway.
  maxSpeed = len(runway)
  if (startIndex >= len(runway) or startIndex < 0 or initSpeed < 0 or initSpeed > maxSpeed or not runway[startIndex]):
    return False
  # {position i : set of speeds for which we can stop from position i}
  memo = {}
  # Base cases, we can stop when a position is not a spike and speed is zero.
  for position in range(len(runway)):
    if runway[position]:
      memo[position] = set([0])
  # Outer loop to go over positions from the last one to the first one
  for position in reversed(range(len(runway))):
    # Skip positions which contain spikes
    if not runway[position]:
      continue
    # For each position, go over all possible speeds
    for speed in range(1, maxSpeed + 1):
      # Recurrence relation is the same as in the recursive version.
      for adjustedSpeed in [speed, speed - 1, speed + 1]:
        if (position + adjustedSpeed in memo and
            adjustedSpeed in memo[position + adjustedSpeed]):
          memo[position].add(speed)
          break
  return initSpeed in memo[startIndex]

In [3]:
def insertIntoMemo(memo, startIndex, initSpeed, value):
    if startIndex not in memo:
        memo[startIndex] = {}
    memo[startIndex][initSpeed] = value
    

def canStopRecursiveWithMemo(runway, initSpeed, startIndex = 0, memo = None):
  # Only done the first time to initialize the memo.
  if memo == None:
    memo = {}
  # First check if the result exists in memo
  if startIndex in memo and initSpeed in memo[startIndex]:
    return memo[startIndex][initSpeed]
  # negative base cases need to go first
  if (startIndex >= len(runway) or startIndex < 0 or initSpeed < 0 or not runway[startIndex]):
    insertIntoMemo(memo, startIndex, initSpeed, False)
    return False
  # base case for a stopping condition
  if initSpeed == 0:
    insertIntoMemo(memo, startIndex, initSpeed, True)
    return True
  # Try all possible paths
  for adjustedSpeed in [initSpeed, initSpeed - 1, initSpeed + 1]:
    # Recurrence relation: If you can stop from any of the subproblems,
    # you can also stop from the main problem
    if canStopRecursiveWithMemo(runway, adjustedSpeed, startIndex + adjustedSpeed, memo):
      insertIntoMemo(memo, startIndex, initSpeed, True)
      return True
  insertIntoMemo(memo, startIndex, initSpeed, False)
  return False

Peter Norvig comment
```
@lru_cache(None)
def canstop(runway, speed, pos=0):
    "Can stop if on the runway and on a non-spike, and either are stopped 
     or can stop after a move."
    return (0 <= pos < len(runway) and runway[pos] and
            (speed == 0 or any(canstop(runway, s, pos + s) 
                               for s in (speed - 1, speed, speed + 1))))
```

__Test__

In [7]:
import random

print(random.randint(0, 5))

5


In [8]:
def fSetupRunway(i_length, pct_spike):
    spike_cnt = int(pct_spike * i_length)
    indx_spike = [random.randint(1,i_length) for i in range(1,spike_cnt)]
    runway = [True] * i_length
    for (index, replacement) in zip(indx_spike, [False]*spike_cnt):
        runway[index] = replacement
    return runway

In [9]:
i_length = 10
pct_spike = .2
initSpeed = 4
runway = fSetupRunway(i_length, pct_spike)

In [10]:
resultRecursive = []
for i in range(0,len(runway)):
    tmp = canStopRecursive(runway, initSpeed, startIndex = i)
    resultRecursive.append(tmp)

In [11]:
resultIterative = []
for i in range(0,len(runway)):
    tmp = canStopIterative(runway, initSpeed, startIndex = i)
    resultIterative.append(tmp)

In [12]:
resultIterativeWithMemo = []
for i in range(0,len(runway)):
    tmp = canStopRecursiveWithMemo(runway, initSpeed, startIndex = i, memo = None)
    resultIterativeWithMemo.append(tmp)

In [13]:
resultIterative == resultIterativeWithMemo

True

In [14]:
resultRecursive == resultIterative

True

In [15]:
resultIterativeWithMemo == resultIterative

True

In [16]:
i_length = 100
pct_spike = .2
initSpeed = 4
runway = fSetupRunway(i_length, pct_spike)

In [17]:
resultRecursive = []
%timeit for i in range(0,len(runway)): tmp = canStopRecursive(runway, initSpeed, startIndex = i); resultRecursive.append(tmp);

511 µs ± 27.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
resultIterative = []
%timeit for i in range(0,len(runway)): tmp = canStopIterative(runway, initSpeed, startIndex = i); resultIterative.append(tmp)

199 ms ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
resultIterativeWithMemo = []
%timeit for i in range(0,len(runway)): tmp = canStopRecursiveWithMemo(runway, initSpeed, startIndex = i, memo = None); resultIterativeWithMemo.append(tmp)

970 µs ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
i_length = 500
pct_spike = .2
initSpeed = 25
runway = fSetupRunway(i_length, pct_spike)

In [21]:
resultRecursive = []
%timeit for i in range(0,len(runway)): tmp = canStopRecursive(runway, initSpeed, startIndex = i); resultRecursive.append(tmp);

36.2 s ± 2.87 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
resultIterative = []
%timeit for i in range(0,len(runway)): tmp = canStopIterative(runway, initSpeed, startIndex = i); resultIterative.append(tmp)

33.8 s ± 2.98 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
resultIterativeWithMemo = []
%timeit for i in range(0,len(runway)): tmp = canStopRecursiveWithMemo(runway, initSpeed, startIndex = i, memo = None); resultIterativeWithMemo.append(tmp)

565 ms ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
